In [2]:
!pip install simpletransformers

In [3]:
!pip install wandb

In [4]:
!wandb login 0d184bb37f834cea42e294066a86c20ffe1a8040

wandb: Appending key for api.wandb.ai to your netrc file: /afs/csail.mit.edu/u/g/getzn/.netrc


In [5]:
import torch
torch.cuda.set_device(1)

In [19]:
#!wget -O lipophilicity.csv https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/Lipophilicity.csv

import pandas as pd
# from datasets import Dataset

# Load the CSV file
df = pd.read_csv('chemberta/data/lipo.csv')[["crippen_logp", "smiles"]]
df.rename(columns = {'crippen_logp':'labels', "smiles": "text"}, inplace = True)
split = int(.80 * len(df))
train_dataset = df[:split]
valid_dataset = df[split:]

# train_dataset = tokenizer(train_dataset['text'].values.tolist(),
#                         add_special_tokens=True,
#                         truncation=True,
#                         max_length=512,
#                         padding=True,
#                         return_tensors='pt'
#                         )
# valid_dataset = tokenizer(valid_dataset['text'].values.tolist(),
#                           add_special_tokens=True,
#                           truncation=True,
#                           max_length=512,
#                           padding=True,
#                           return_tensors='pt')
train_dataset

,labels,text
0,3.54890,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14
1,2.99010,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...
2,3.67390,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl
3,1.92370,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...
4,2.00690,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...
...,...,...
3355,3.41654,C[C@H](COc1cn2ncnc(Oc3ccc4[nH]c(C)cc4c3F)c2c1C...
3356,2.60440,Fc1ccc(CN2CCN(CC2)c3ccc4nnc(n4n3)C(F)(F)F)cc1
3357,1.77700,Oc1c2C(=O)NNC(=O)c2nc3cc(Cl)cc(Cl)c13
3358,5.37270,CCN(CC)C(=O)c1ccc(cc1)C(=C2CCNCC2)c3cccc(c3)C(...


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [3]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=1)

from simpletransformers.classification import ClassificationModel

import os

from sklearn.metrics import r2_score
import wandb
def r2_metric(labels, preds):
    # Calculate R-squared score
    r2 = r2_score(labels, preds)
    # Log the score to wandb
    wandb.log({'r2_score': r2})
    return r2

def train_and_eval_model(model_name, tokenizer_name):
    model_path = os.path.join("chemberta/saved_models", model_name, "final")
    if tokenizer == "default":
        tokenizer_path = "seyonec/SMILES_tokenized_PubChem_shard00_160k"
    else:
        tokenizer_path = os.path.join("chemberta/saved_tokenizers", tokenizer_name)
    
    for i, (train_idxs, test_idxs) in enumerate(kf.split(df)):
        print(f"Training model {model_name} on test split {i}")
        train_dataset = df.iloc[train_idxs]
        valid_dataset = df.iloc[test_idxs]
        
        model = ClassificationModel(
            "roberta",
            model_path,
            num_labels=1,
            args={
                "regression": True,
                'wandb_project': "Finetuning Models with 5-Fold Cross-Validation",
                "lazy_text_column": 1,
                "lazy_labels_column": 0,
                "use_cuda": True,
                "cuda_device": 0,
                "evaluate_during_training": True,
                "evaluate_during_training_steps": 50,
                "num_train_epochs": 5,
                "overwrite_output_dir": True,
                "wandb_kwargs": {
                    "project": "Finetuning ChemBERTa Models with 5-Fold Cross-Validation",
                    "name": f"{model_name} on Lipophilicity split {i}"
                },
            },
                tokenizer_type = 'auto',
                tokenizer_name = tokenizer_path
            )
        model.train_model(
            train_dataset,
            eval_df = valid_dataset,
            pearsonr=r2_metric
            )
        model.eval_model(valid_dataset)


2023-12-15 21:15:11.912919: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-15 21:15:14.351625: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 21:15:17.457622: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/data/scratch/getzn/miniconda/envs/new_chemberta_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autono

In [ ]:
import multiprocessing
multiprocessing.set_start_method(None)

In [ ]:
model_name = "mlm_100k_zinc_crippen_balanced"
tokenizer_name = "default"

model_path = os.path.join("chemberta/saved_models", model_name, "final")
if tokenizer_name == "default":
    tokenizer_path = "seyonec/SMILES_tokenized_PubChem_shard00_160k"
else:
    tokenizer_path = os.path.join("chemberta/saved_tokenizers", tokenizer_name)

for i, (train_idxs, test_idxs) in enumerate(kf.split(df)):
    print(f"Training model {model_name} on test split {i}")
    train_dataset = df.iloc[train_idxs]
    valid_dataset = df.iloc[test_idxs]

    model = ClassificationModel(
        "roberta",
        "chemberta/saved_models/mlm_100k_zinc_crippen_balanced_smiles_v2_large/final",
        num_labels=1,
        args={
            "regression": True,
            'wandb_project': 'project-name',
            "lazy_text_column": 1,
            "lazy_labels_column": 0,
            "use_cuda": True,
            "cuda_device": 1,
            "evaluate_during_training": True,
            "evaluate_during_training_steps": 50,
            "num_train_epochs": 5,
            "overwrite_output_dir": True
        },
        tokenizer_type = 'auto',
        tokenizer_name = "chemberta/saved_tokenizers/smiles_tokenizer3"
    )
    print("device", model.device)
    
    # model = ClassificationModel(
    #     "roberta",
    #     model_path,
    #     num_labels=1,
    #     args={
    #         "regression": True,
    #         'wandb_project': "Finetuning Models with 5-Fold Cross-Validation",
    #         "lazy_text_column": 1,
    #         "lazy_labels_column": 0,
    #         "use_cuda": True,
    #         "cuda_device": 0,
    #         "evaluate_during_training": True,
    #         "evaluate_during_training_steps": 50,
    #         "num_train_epochs": 5,
    #         "overwrite_output_dir": True,
    #         "wandb_kwargs": {
    #             "project": "Finetuning ChemBERTa Models with 5-Fold Cross-Validation",
    #             "name": f"{model_name} on Lipophilicity split {i}"
    #         },
    #     },
    #         tokenizer_type = 'auto',
    #         tokenizer_name = tokenizer_path
    #     )
    print(train_dataset)
    raise AssertionError
    model.train_model(
        train_dataset,
        eval_df = valid_dataset,
        r2=r2_metric,
        
        )
    model.eval_model(valid_dataset)

In [8]:
model.device

NameError: name 'model' is not defined

In [ ]:
model.train_model(
        train_dataset,
        eval_df = valid_dataset,
        pearsonr=r2_metric
        )

In [ ]:
models = [
    "mlm_100k_zinc_crippen_balanced",
    "mlm_100k_zinc_crippen_uniform"
]
tokenizers = ["default", "default"]
for model, tokenizer in zip(models, tokenizers):
    train_and_eval_model(model, tokenizer)

In [20]:
from simpletransformers.classification import ClassificationModel
from scipy import stats
from sklearn.metrics import r2_score
import wandb
def r2_metric(labels, preds):
    # Calculate R-squared score
    r2 = r2_score(labels, preds)
    # Log the score to wandb
    wandb.log({'r2_score': r2})
    return r2

def pearsonr_metric(labels, preds):
    pearsonr = stats.pearsonr(labels, preds)[0]
    
    wandb.log({'pearsonr': pearsonr})

    return pearsonr
def spearmanr_metric(labels, preds):
    spearmanr = stats.spearmanr(labels,preds).statistic
    
    wandb.log({'spearmanr': spearmanr})

    return spearmanr


def finetune(model_path, tokenizer_path, run_name):
    model = ClassificationModel(
        "roberta",
        model_path,
        num_labels=1,
        args={
            "regression": True,
            'wandb_project': 'Finetuning-with-crippenlogp',
            "lazy_text_column": 1,
            "lazy_labels_column": 0,
            "use_cuda": True,
            "cuda_device": 0,
            "evaluate_during_training": True,
            "evaluate_during_training_steps": 50,
            "num_train_epochs": 5,
            "overwrite_output_dir": True,
            "wandb_kwargs": {
                    "name": run_name},},
            tokenizer_type = 'auto',
            tokenizer_name = tokenizer_path

    )

    model.train_model(
    train_dataset,
    eval_df = valid_dataset,
    r2=r2_metric,
    pearsonr = pearsonr_metric,
    spearmanr = spearmanr_metric
    )


In [ ]:
token_path = "seyonec/SMILES_tokenized_PubChem_shard00_160k"
model_paths = ["chemberta/saved_models/final_train_zinc_100k_crippen_balanced_50_bins_eval_lipo/final", 
              "chemberta/saved_models/final_train_zinc_100k_crippen_matched_50_bins_eval_lipo/final", 
              "chemberta/saved_models/final_train_zinc_100k_uniform_eval_lipo/final"]
run_name = ["balanced_50_bins", "matched_50_bins", "uniform"]

for i,model in enumerate(model_paths):
    finetune(model, token_path, run_name[i])


In [ ]:
model.eval_model(valid_dataset)

In [8]:
model_path = "chemberta/saved_models/final_train_zinc_100k_crippen_balanced_50_bins_eval_lipo/final"
tokenizer_path = "seyonec/SMILES_tokenized_PubChem_shard00_160k"
model = ClassificationModel(
        "roberta",
        model_path,
        num_labels=1,
        args={
            "regression": True,
            'wandb_project': 'Finetuning-with-crippenlogp',
            "lazy_text_column": 1,
            "lazy_labels_column": 0,
            "use_cuda": True,
            "cuda_device": 0,
            "evaluate_during_training": True,
            "evaluate_during_training_steps": 50,
            "num_train_epochs": 5,
            "overwrite_output_dir": True,
            "wandb_kwargs": {
                    "name": "test run ignore"},},
            tokenizer_type = 'auto',
            tokenizer_name = tokenizer_path

    )

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at chemberta/saved_models/final_train_zinc_100k_crippen_balanced_50_bins_eval_lipo/final and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model.loss_fct